# [Quickstrart](https://tutorials.pytorch.kr/beginner/basics/quickstart_tutorial.html)


1. [데이터 작업](#1-working-with-data)
2. [모델 만들기](#2-creating-models)
3. [모델 매개변수 최적화 하기](#3-optimizing-the-model-parameters)
4. [모델 저장 및 불러오기](#4-saving-and-loading-models)
5. [모델 사용하기](#5-predict-with-models)

## 1. Working with Data 

#### 데이터 작업을 위한 기본 요소
- torch.utils.data.Dataset에는 샘플과 정답(label)을 저장
- torch.utils.data.DataLoader는 Dataset을 순회가능한 객체(iterable)로 감쌈

torchvision.datasets에는 CIFAR, COCO 등 다양한 실제 vision 데이터에 대한 dataset을 포함하고 있음. 이 튜토리얼에서는 FashionMNIST를 사용. 모든 torchvision dataset은 샘플과 정답을 각각 변경하기 위한 transform과 target_transform 두 인자를 포함

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

/home/doeun/anaconda3/envs/HF/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_cuda.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
training_data = datasets.FashionMNIST(
    root = 'data',
    train = True,
    download = True,
    transform = ToTensor()
)

test_data = datasets.FashionMNIST(
    root = 'data',
    train = False,
    download = True,
    transform = ToTensor()
)

Dataset을 DataLoader의 인자로 전달
- 데이터셋을 iterable로 감쌈
- 자동화된 batch, sampling shuffle 및 multiprocess dataloading 지원

batch size = 64 : dataloader 객채의 각 요소는 64개의 feature과 정답(label)을 묶음(batch)로 반환

In [3]:
batch_size = 64 

print(f'len of dataset : {len(training_data)}')

train_dataloader = DataLoader(training_data, batch_size= batch_size)
test_dataloader = DataLoader(test_data, batch_size= batch_size)

print(f'len of dataloader : {len(train_dataloader)}')

for X, y in test_dataloader:
    print(f'Shape of X [N, C, H, W] : {X.shape}, {X.dtype}')
    print(f'Shape of y : {y.shape}, {y.dtype}')
    break

len of dataset : 60000
len of dataloader : 938
Shape of X [N, C, H, W] : torch.Size([64, 1, 28, 28]), torch.float32
Shape of y : torch.Size([64]), torch.int64


## 2. Creating Models

신경망 모델
- nn.Module을 상속받는 class를 생성하여 정의
- \_\_init\_\_ 에서 신경망의 layer 정의
- forward에서 신경망에 데이터를 어떻게 전달할지 지정
- 가능한 경우 GPU나 MPS로 신경망을 이동시켜 연산을 가속


In [4]:
device = (
    'cuda' if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f'using {device}')

using cpu


In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10) 
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## 3. Optimizing the Model Parameters 

모델 학습에는 loss function과 optimizer가 필요

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=1e-3)

각 학습 단계에서 모델은 
- batch로 묶여서 제공되는 학습 데이터셋에 대한 예측을 수행
- 예측 오류를 역전파 하여 모델의 매개변수를 조정

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for n, (X,y) in enumerate(dataloader) :
        X, y = X.to(device), y.to(device)
        
        #예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred,y)
        
        #역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if n % 250 == 0 :
            loss, current = loss.item(), (n+1) * len(X)
            print(f'loss : {loss:>7f}, [{current:>5d}/{size:>5d}]')

모델이 학습하고 있는지 확인하기 위해 테스트 데이터셋으로 모델의 성능 확인

In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f'Test Error : \n Accuracy :{(100*correct):>0.1f}%, Avg loss : {test_loss:>8f}\n')

학습 단계는 여러번의 반복 단계(epochs)를 거쳐서 수행
- 각 epoch에서는 모델이 더 나은 예측을 하기 위해 매개변수를 학습
- 각 epoch마다 모델의 accuracy와 loss를 출력
- epoch 마다 정확도가 증가하고 손실이 감소하는 것을 보려고 함

In [9]:
epochs = 10

for t in range(epochs):
    print(f'Epoch {t+1}\n-------------------------')
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn) 

print("Done!")   

Epoch 1
-------------------------
loss : 2.293193, [   64/60000]
loss : 2.257958, [16064/60000]
loss : 2.219961, [32064/60000]
loss : 2.182379, [48064/60000]
Test Error : 
 Accuracy :46.1%, Avg loss : 2.162863

Epoch 2
-------------------------
loss : 2.166061, [   64/60000]
loss : 2.119100, [16064/60000]
loss : 2.018646, [32064/60000]
loss : 1.947069, [48064/60000]
Test Error : 
 Accuracy :61.9%, Avg loss : 1.904841

Epoch 3
-------------------------
loss : 1.932667, [   64/60000]
loss : 1.863440, [16064/60000]
loss : 1.677570, [32064/60000]
loss : 1.558992, [48064/60000]
Test Error : 
 Accuracy :60.0%, Avg loss : 1.540212

Epoch 4
-------------------------
loss : 1.605775, [   64/60000]
loss : 1.553874, [16064/60000]
loss : 1.367319, [32064/60000]
loss : 1.227922, [48064/60000]
Test Error : 
 Accuracy :62.7%, Avg loss : 1.270327

Epoch 5
-------------------------
loss : 1.349178, [   64/60000]
loss : 1.324568, [16064/60000]
loss : 1.162399, [32064/60000]
loss : 1.010567, [48064/60000

## 4. Saving and Loading Models 

### 모델 저장하기

In [10]:
import os
path, dir_name, model_name = os.getcwd(),"model", "fashion.pt"
model_path = os.path.join(path,dir_name)
if not os.path.exists(model_path): os.mkdir(model_path)
model_path = os.path.join(model_path,model_name)

모델 저장하는 일반적인 방법은, (모델의 매개변수들을 포함하여) 내부 상태 사전(internal state dictionary)를 직렬화(serialize)하는 것

저장 torch.save(object,path)
- model : 전체 모델 저장, *.pt
- model.state_dict() : 모델 객체의 state_dict 저장, *.pt
- {'model':model.state_dict(), 'optimizer':optimizer.state_dict()} : 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar 값도 포함하여 저장, *.tar

In [11]:
torch.save(model.state_dict(),model_path)
print(f'saved pytorch model state to {model_path}')

saved pytorch model state to /home/doeun/code/AI/pytorch_tutorial/model/fashion.pt


### 모델 불러오기

모델을 불러오는 과정에는 모델 구조를 다시 만들고 상태 사전을 모델에 불러오는 과정 포함

In [12]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

## 5. Predict with Models 


In [13]:

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'predicted : {predicted}, actual : {actual}')

predicted : Ankle boot, actual : Ankle boot
